# Extract GLORYS12V1 data for NW Atlantic region
Created by Ivan Lima on Tue May  3 2022 16:51:50 -0400

In [ ]:
%matplotlib inline
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, datetime
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

## OPeNDAP/CMEM settings

In [ ]:
from pydap.client import open_url
# from pydap.cas.urs import setup_session
from pydap.cas.get_cookies import setup_session

creds = open('.cmems_creds').readlines()
username = creds[0].strip()
password = creds[1].strip()

cas_url = 'https://cmems-cas.cls.fr/cas/login'
session = setup_session(cas_url, username, password)
session.cookies.set('CASTGC', session.cookies.get_dict()['CASTGC'])

## Monthly data

In [ ]:
url = 'https://my.cmems-du.eu/thredds/dodsC/cmems_mod_glo_phy_my_0.083_P1M-m'
store = xr.backends.PydapDataStore.open(url, session=session)
# store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8'))

ybeg, yend = 1993, 2020
start_str, end_str = '{}-01-16T12:00:00.000000000', '{}-12-16T12:00:00.000000000'
start_yr = [start_str.format(y) for y in range(ybeg, yend)]
end_yr = [end_str.format(y) for y in range(ybeg, yend)]

outdir = '/bali/data/ilima/GLORYS12V1/monthly'
with xr.open_dataset(store, drop_variables=['sithick', 'siconc', 'usi', 'vsi']) as ds:
    for start, end in zip(start_yr, end_yr):
        ds_yr = ds.sel(time=slice(start, end), latitude=slice(34, 46), longitude=slice(-78, -64))
        ds_yr.load()
        filename = 'GLORYS12V1_NW_Atlantic_{}_monthly.nc'.format(ds_yr.time.dt.year[0].item())
        outfile = os.path.join(outdir, filename)
        print('writing {}'.format(outfile))
        ds_yr.to_netcdf(outfile, 'w')

In [ ]:
# ds.zos.sel(time='1993-01-16T12:00:00.000000000', latitude=slice(34,46), longitude=slice(-78, -64)).plot()

## Daily data

In [ ]:
url = 'https://my.cmems-du.eu/thredds/dodsC/cmems_mod_glo_phy_my_0.083_P1D-m' # daily data
store = xr.backends.PydapDataStore.open(url, session=session)
# store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8'))

ybeg, yend = 1993, 2020
start_str, end_str = '{}-01-01T12:00:00.000000000', '{}-12-31T12:00:00.000000000'
start_yr = [start_str.format(y) for y in range(ybeg, yend)]
end_yr = [end_str.format(y) for y in range(ybeg, yend)]

outdir = '/bali/data/ilima/GLORYS12V1/daily'
with xr.open_dataset(store, drop_variables=['sithick', 'siconc', 'usi', 'vsi', 'uo', 'vo']) as ds:
    for start, end in zip(start_yr, end_yr):
        ds_yr = ds.sel(time=slice(start, end), latitude=slice(34, 46), longitude=slice(-78, -64))
        ds_yr.load()
        filename = 'GLORYS12V1_NW_Atlantic_{}_daily.nc'.format(ds_yr.time.dt.year[0].item())
        outfile = os.path.join(outdir, filename)
        print('writing {}'.format(outfile))
        ds_yr.to_netcdf(outfile, 'w')